In [1]:
import pandas as pd

data = pd.read_json("./05_clean_data.json.zip")
data

,instance,#Variables,#Clauses,iteration_info,baseline_alg,initial_sample_path,initial_sample_size,lower_bound,optimized_sample_size
0,email,10,17,"[{'nbrhd_tuples': 70, 'nbrhd_confs': 8, 'itera...",FIDE-YASA(m=1),2023-03-01_13-51-03/3_1_3_5_sample.csv,8,6,6
1,ChatClient,14,20,"[{'nbrhd_tuples': 176, 'nbrhd_confs': 10, 'ite...",FIDE-YASA(m=1),2023-03-01_13-51-03/4_1_3_1_sample.csv,10,7,7
2,ChatClient,14,20,"[{'nbrhd_tuples': 176, 'nbrhd_confs': 10, 'ite...",FIDE-YASA(m=1),2023-03-01_13-51-03/4_1_3_2_sample.csv,10,7,7
3,ChatClient,14,20,"[{'nbrhd_tuples': 176, 'nbrhd_confs': 10, 'ite...",FIDE-YASA(m=1),2023-03-01_13-51-03/4_1_3_3_sample.csv,10,7,7
4,ChatClient,14,20,"[{'nbrhd_tuples': 176, 'nbrhd_confs': 10, 'ite...",FIDE-YASA(m=1),2023-03-01_13-51-03/4_1_3_4_sample.csv,10,7,7
...,...,...,...,...,...,...,...,...,...
1933,berkeleyDB2,119,346,"[{'nbrhd_tuples': 198, 'nbrhd_confs': 10, 'ite...",Incling,2023-03-01_13-51-03/20_1_2_5_sample.csv,28,11,12
1934,soletta_2015-06-26_18-38-56,129,192,"[{'nbrhd_tuples': 223, 'nbrhd_confs': 22, 'ite...",Incling,2023-03-01_13-51-03/21_1_2_1_sample.csv,44,24,24
1935,soletta_2015-06-26_18-38-56,129,192,"[{'nbrhd_tuples': 227, 'nbrhd_confs': 22, 'ite...",Incling,2023-03-01_13-51-03/21_1_2_2_sample.csv,44,24,24
1936,soletta_2015-06-26_18-38-56,129,192,"[{'nbrhd_tuples': 193, 'nbrhd_confs': 21, 'ite...",Incling,2023-03-01_13-51-03/21_1_2_3_sample.csv,44,24,24


In [2]:
# Find the best values for each instance as reference.
best_solutions = (
    data[["instance", "initial_sample_size", "optimized_sample_size"]]
    .groupby("instance")
    .min()
    .reset_index()
    .rename(
        columns={
            "initial_sample_size": "best_baseline",
            "optimized_sample_size": "best_lns_ub",
        }
    )
)
best_lb = (
    data[["instance", "lower_bound"]]
    .groupby("instance")
    .max()
    .reset_index()
    .rename(columns={"lower_bound": "best_lb"})
)
mean_ub = (
    data[["instance", "optimized_sample_size"]]
    .groupby("instance")
    .mean()
    .reset_index()
    .rename(columns={"optimized_sample_size": "mean_lns_ub"})
)
best_values = best_solutions.merge(best_lb, left_on="instance", right_on="instance")
best_values = best_values.merge(mean_ub, left_on="instance", right_on="instance")

In [3]:
samplns_yasa = (
    data[data["baseline_alg"] == "YASA(m=1)"][["instance", "optimized_sample_size"]]
    .groupby("instance")
    .mean()
    .reset_index()
    .rename(columns={"optimized_sample_size": "samplns_yasa"})
)
best_values = best_values.merge(samplns_yasa, left_on="instance", right_on="instance")
best_values

,instance,best_baseline,best_lns_ub,best_lb,mean_lns_ub,samplns_yasa
0,APL,9,7,7,7.000000,7.0
1,APL-Model,10,8,8,8.000000,8.0
2,BankingSoftware,40,29,28,29.000000,29.0
3,BattleofTanks,444,331,256,344.346939,342.2
4,ChatClient,8,7,7,7.000000,7.0
5,DMIE,25,16,16,16.152174,16.2
6,E-Shop,20,12,9,12.446809,12.2
7,EMBToolkit,1883,1876,1872,1947.322581,1877.4
8,FameDB,8,8,8,8.000000,8.0
9,FeatureIDE,11,8,7,8.000000,8.0


In [4]:
data = data.merge(best_values, left_on="instance", right_on="instance")

In [5]:
t = data[data["initial_sample_size"] == data["best_baseline"]].drop_duplicates(
    subset=["instance"], keep="last"
)

In [6]:
t

,instance,#Variables,#Clauses,iteration_info,baseline_alg,initial_sample_path,initial_sample_size,lower_bound,optimized_sample_size,best_baseline,best_lns_ub,best_lb,mean_lns_ub,samplns_yasa
52,email,10,17,"[{'nbrhd_tuples': 70, 'nbrhd_confs': 6, 'itera...",FIDE-YASA(m=5),2023-03-01_13-51-03/3_1_5_4_sample.csv,6,6,6,6,6,6,6.000000,6.0
106,ChatClient,14,20,"[{'nbrhd_tuples': 176, 'nbrhd_confs': 8, 'iter...",FIDE-YASA(m=10),2023-03-01_13-51-03/4_1_6_5_sample.csv,8,7,7,8,7,7,7.000000,7.0
158,toybox_2006-10-31_23-30-06,16,13,"[{'nbrhd_tuples': 208, 'nbrhd_confs': 7, 'iter...",YASA(m=10),2023-03-01_13-51-03/5_1_10_5_sample.csv,9,8,8,9,8,8,8.000000,8.0
194,fiasco_2020-12-01_14-09-14,258,1542,"[{'nbrhd_tuples': 245, 'nbrhd_confs': 25, 'ite...",FIDE-YASA(m=10),2023-03-01_13-51-03/25_1_6_5_sample.csv,214,196,197,214,196,196,197.111111,197.6
250,E-Shop,326,499,"[{'nbrhd_tuples': 213, 'nbrhd_confs': 3, 'iter...",FIDE-YASA(m=10),2023-03-01_13-51-03/28_1_6_5_sample.csv,20,8,12,20,12,9,12.446809,12.2
303,calculate,9,15,"[{'nbrhd_tuples': 38, 'nbrhd_confs': 9, 'itera...",FIDE-YASA(m=5),2023-03-01_13-51-03/1_1_5_5_sample.csv,9,5,5,9,5,5,5.000000,5.0
352,berkeleyDB1,76,147,"[{'nbrhd_tuples': 188, 'nbrhd_confs': 9, 'iter...",YASA(m=10),2023-03-01_13-51-03/17_1_10_2_sample.csv,19,15,15,19,15,15,15.000000,15.0
404,gpl,38,99,"[{'nbrhd_tuples': 240, 'nbrhd_confs': 12, 'ite...",YASA(m=10),2023-03-01_13-51-03/13_1_10_4_sample.csv,18,16,16,18,16,16,16.000000,16.0
419,SortingLine,39,77,"[{'nbrhd_tuples': 179, 'nbrhd_confs': 7, 'iter...",FIDE-YASA(m=10),2023-03-01_13-51-03/14_1_6_5_sample.csv,12,9,9,12,9,9,9.000000,9.0
504,dell,46,244,"[{'nbrhd_tuples': 238, 'nbrhd_confs': 20, 'ite...",YASA(m=10),2023-03-01_13-51-03/15_1_10_4_sample.csv,34,31,31,34,31,31,31.000000,31.0


In [7]:
t["baseline"] = t.apply(
    (lambda row: f"{row['initial_sample_size']}/{row['baseline_alg']}"), axis=1
)
t["sampLNS"] = t.apply(
    (lambda row: f"{row['best_lns_ub']} (\\O {int(round(row['mean_lns_ub']))})"), axis=1
)


def shorten(s):
    if len(s) > 23:
        return s[:20] + "..."
    return s


# t["instance_details"] = t.apply((lambda row: f"{shorten(row['instance'])} ({row['#Variables']}/{row['#Clauses']})"), axis=1)
t["instance_details"] = t.apply((lambda row: f"{shorten(row['instance'])}"), axis=1)
t["reduction"] = 100 * (1 - t["samplns_yasa"] / t["best_baseline"])
t["rel_size_after_samplns"] = 100 * (t["samplns_yasa"] / t["best_baseline"])
t["savings"] = 100 * (1 - t["samplns_yasa"] / t["best_baseline"])
t["opt_1"] = t["best_baseline"] == t["best_lb"]
t["opt_2"] = t["samplns_yasa"].apply(lambda x: round(x)) == t["best_lb"]
t

,instance,#Variables,#Clauses,iteration_info,baseline_alg,initial_sample_path,initial_sample_size,lower_bound,optimized_sample_size,best_baseline,...,mean_lns_ub,samplns_yasa,baseline,sampLNS,instance_details,reduction,rel_size_after_samplns,savings,opt_1,opt_2
52,email,10,17,"[{'nbrhd_tuples': 70, 'nbrhd_confs': 6, 'itera...",FIDE-YASA(m=5),2023-03-01_13-51-03/3_1_5_4_sample.csv,6,6,6,6,...,6.000000,6.0,6/FIDE-YASA(m=5),6 (\O 6),email,0.000000,100.000000,0.000000,True,True
106,ChatClient,14,20,"[{'nbrhd_tuples': 176, 'nbrhd_confs': 8, 'iter...",FIDE-YASA(m=10),2023-03-01_13-51-03/4_1_6_5_sample.csv,8,7,7,8,...,7.000000,7.0,8/FIDE-YASA(m=10),7 (\O 7),ChatClient,12.500000,87.500000,12.500000,False,True
158,toybox_2006-10-31_23-30-06,16,13,"[{'nbrhd_tuples': 208, 'nbrhd_confs': 7, 'iter...",YASA(m=10),2023-03-01_13-51-03/5_1_10_5_sample.csv,9,8,8,9,...,8.000000,8.0,9/YASA(m=10),8 (\O 8),toybox_2006-10-31_23...,11.111111,88.888889,11.111111,False,True
194,fiasco_2020-12-01_14-09-14,258,1542,"[{'nbrhd_tuples': 245, 'nbrhd_confs': 25, 'ite...",FIDE-YASA(m=10),2023-03-01_13-51-03/25_1_6_5_sample.csv,214,196,197,214,...,197.111111,197.6,214/FIDE-YASA(m=10),196 (\O 197),fiasco_2020-12-01_14...,7.663551,92.336449,7.663551,False,False
250,E-Shop,326,499,"[{'nbrhd_tuples': 213, 'nbrhd_confs': 3, 'iter...",FIDE-YASA(m=10),2023-03-01_13-51-03/28_1_6_5_sample.csv,20,8,12,20,...,12.446809,12.2,20/FIDE-YASA(m=10),12 (\O 12),E-Shop,39.000000,61.000000,39.000000,False,False
303,calculate,9,15,"[{'nbrhd_tuples': 38, 'nbrhd_confs': 9, 'itera...",FIDE-YASA(m=5),2023-03-01_13-51-03/1_1_5_5_sample.csv,9,5,5,9,...,5.000000,5.0,9/FIDE-YASA(m=5),5 (\O 5),calculate,44.444444,55.555556,44.444444,False,True
352,berkeleyDB1,76,147,"[{'nbrhd_tuples': 188, 'nbrhd_confs': 9, 'iter...",YASA(m=10),2023-03-01_13-51-03/17_1_10_2_sample.csv,19,15,15,19,...,15.000000,15.0,19/YASA(m=10),15 (\O 15),berkeleyDB1,21.052632,78.947368,21.052632,False,True
404,gpl,38,99,"[{'nbrhd_tuples': 240, 'nbrhd_confs': 12, 'ite...",YASA(m=10),2023-03-01_13-51-03/13_1_10_4_sample.csv,18,16,16,18,...,16.000000,16.0,18/YASA(m=10),16 (\O 16),gpl,11.111111,88.888889,11.111111,False,True
419,SortingLine,39,77,"[{'nbrhd_tuples': 179, 'nbrhd_confs': 7, 'iter...",FIDE-YASA(m=10),2023-03-01_13-51-03/14_1_6_5_sample.csv,12,9,9,12,...,9.000000,9.0,12/FIDE-YASA(m=10),9 (\O 9),SortingLine,25.000000,75.000000,25.000000,False,True
504,dell,46,244,"[{'nbrhd_tuples': 238, 'nbrhd_confs': 20, 'ite...",YASA(m=10),2023-03-01_13-51-03/15_1_10_4_sample.csv,34,31,31,34,...,31.000000,31.0,34/YASA(m=10),31 (\O 31),dell,8.823529,91.176471,8.823529,False,True


In [8]:
# Code to query the time of the last change in lower or upper bound.


# Collect the data from the iteration_info events.
class EventCollector:
    def __init__(self):
        self.events = {
            "time": [],
            "val": [],
            "type": [],
            "instance": [],
            "path": [],
            "alg": [],
            "initial_sample_size": [],
            "y": [],
            "final": [],
        }

    def __call__(self, row):
        it_data = row["iteration_info"]

        def add(val, lbub, time, final=False):
            if time > 900:
                return
            self.events["time"].append(time / 60)
            self.events["val"].append(val)
            self.events["type"].append(lbub)
            self.events["instance"].append(row["instance"])
            self.events["alg"].append(row["baseline_alg"])
            self.events["path"].append(row["initial_sample_path"])
            self.events["initial_sample_size"].append(row["initial_sample_size"])
            self.events["y"].append(100 * (val / row["best_lb"]))
            self.events["final"].append(final)

        # add a zero entry
        add(0, "Lower", 0)
        add(row["initial_sample_size"], "Upper", 0)
        for event in it_data:
            add(
                event["lb"],
                "Lower",
                event["time"],
                final=event["lb"] == row["lower_bound"],
            )
            add(
                event["ub"],
                "Upper",
                event["time"],
                final=event["ub"] == row["optimized_sample_size"],
            )


instance_infos = (
    data[["instance", "#Variables", "#Clauses"]]
    .drop_duplicates()
    .sort_values(by=["#Variables", "#Clauses"])
)

ec = EventCollector()
data[data["baseline_alg"] == "FIDE-YASA(m=1)"].sort_values(
    by=["#Variables", "#Clauses"]
).apply(ec, axis=1)
t_events = pd.DataFrame(ec.events)
t_ = t_events.groupby(["instance", "path"])["time"].max().reset_index()
unsolved_instances = t_[t_["time"] == 0]["instance"].unique().tolist()
solved_instances = [
    x for x in instance_infos["instance"].tolist() if x not in unsolved_instances
]

time_until_last_change = (
    t_events[t_events["final"] & (t_events["instance"].isin(solved_instances))]
    .groupby(["instance", "path", "type"])[["time"]]
    .min()
    .reset_index()
    .groupby(["instance", "type"])[["time"]]
    .mean()
)

unsolved_symbol = "$*$"


def get_time_info(instance):
    d = time_until_last_change.groupby("instance")["time"].max().to_dict()
    if instance not in d:
        return unsolved_symbol
    else:
        secs = round(d[instance] * 60)
        if secs == 0:
            return "$<\\SI{1}{\\second}$"
        return "\\SI{" + str(secs) + "}{\\second}"

In [9]:
tex_table = ""


def append(*args):
    global tex_table
    tex_table += f"{' '.join(str(x) for x in args)}\n"


t.sort_values(by=["#Variables", "#Clauses"], inplace=True)
for idx in t.index:
    unsolved = t["instance"][idx] in unsolved_instances
    append(
        t["instance_details"][idx].replace("_", "\\_"),
        "& \\num{",
        t["#Variables"][idx],
        "} & \\num{",
        t["#Clauses"][idx],
        "} &",
        ("\\textbf{" if t["opt_1"][idx] else "{") + "\\num[text-series-to-math=true]{",
        t["best_baseline"][idx],
        " }} &",
        unsolved_symbol
        if unsolved
        else (
            ("\\textbf{" if t["opt_2"][idx] else "{")
            + "\\num[text-series-to-math=true]{"
            + str(int(round(t["samplns_yasa"][idx])))
            + "}}"
        ),
        " & \\num{",
        round(t["best_lb"][idx]),
        "} &",
        unsolved_symbol
        if unsolved
        else "\\SI{" + f"{round(t['savings'][idx], 2):.2g}" + "}{\\percent}",
        "&",
        get_time_info(t["instance"][idx]),
        "\\\\",
    )

print(tex_table)

calculate & \num{ 9 } & \num{ 15 } & {\num[text-series-to-math=true]{ 9  }} & \textbf{\num[text-series-to-math=true]{5}}  & \num{ 5 } & \SI{44}{\percent} & $<\SI{1}{\second}$ \\
lcm & \num{ 9 } & \num{ 16 } & {\num[text-series-to-math=true]{ 8  }} & \textbf{\num[text-series-to-math=true]{6}}  & \num{ 6 } & \SI{25}{\percent} & $<\SI{1}{\second}$ \\
email & \num{ 10 } & \num{ 17 } & \textbf{\num[text-series-to-math=true]{ 6  }} & \textbf{\num[text-series-to-math=true]{6}}  & \num{ 6 } & \SI{0}{\percent} & $<\SI{1}{\second}$ \\
ChatClient & \num{ 14 } & \num{ 20 } & {\num[text-series-to-math=true]{ 8  }} & \textbf{\num[text-series-to-math=true]{7}}  & \num{ 7 } & \SI{12}{\percent} & \SI{10}{\second} \\
toybox\_2006-10-31\_23... & \num{ 16 } & \num{ 13 } & {\num[text-series-to-math=true]{ 9  }} & \textbf{\num[text-series-to-math=true]{8}}  & \num{ 8 } & \SI{11}{\percent} & \SI{6}{\second} \\
car & \num{ 16 } & \num{ 33 } & {\num[text-series-to-math=true]{ 6  }} & \textbf{\num[text-series-t

In [10]:
len(t[t["best_baseline"] / t["best_lb"] <= 1.1])

10